<a href="https://colab.research.google.com/github/alexk2206/tds_capstone/blob/Domi-DEV/Productive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Productive Notebook

In [ ]:
!pip install evaluate
!pip install --upgrade sympy

In [ ]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import urllib
from itertools import chain, combinations
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments
import torch
import requests
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer

### Preprocess dataset

Here we split the QA-dataset into train and validation dataset.
Additionnaly, we prepare the dataset to later be useful for response-generation and fine-tuning of a model

❎ Please insert code: load dataset into variable data ❎

In [ ]:
# Example dataset
data = q

# Convert to DataFrame for easy handling
df = pd.DataFrame(data)

# Map the intended answer to the index of the option
df['label'] = df.apply(lambda x: x['options'].index(x['intended_answer']), axis=1)
df['stratify_key'] = df['context_type'] + '_' + df['type']

# Stratified Train-Validation Split
train_df, val_df = train_test_split(
    df,
    train_size=0.8,
    stratify=df['stratify_key'],
    random_state=42
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

NameError: name 'q' is not defined

### Fine-tuning a model


In [ ]:
def fine_tune_model(train_dataset, val_dataset, tokenizer, model):
    # Define training arguments
    training_args = TrainingArguments("trainer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        load_best_model_at_end=True
    )
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

    # Define Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Train the model
    trainer.train()

def compute_metrics(eval_preds, pretrained_dataset_name):
    metric = evaluate.load("glue", pretrained_dataset_name)
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def tokenize_function(example):
    tokenized = tokenizer(
        [example["context"]] * len(example["options"]),  # Repeat context for each option
        [example["question"] + " " + option for option in example["options"]],  # Pair with each option
        truncation=True,
        max_length=512,
        padding="max_length",  # Ensure uniform input length
        return_tensors="pt"
    )

    # Get the label (index of the intended answer)
    if example["type"] == "SINGLE_SELECT":
        # For single-select questions, just find the index of the correct option
        labels = example["options"].index(example["intended_answer"])
    elif example["type"] == "MULTIPLE_SELECT":
        # For multiple-select questions, create binary labels for each option
        labels = [1 if option in example["intended_answer"] else 0 for option in example["options"]]
    else:
        raise ValueError(f"Unknown question type: {example['type']}")

    tokenized["labels"] = labels  # Add labels to the tokenized output
    return tokenized